Possible Paper Titles

1) Improving LLM conversational recommendations: How Empathy Leads to Information Gain

2) SwIG: Sampling with Information Gain to improve conversational recommendations

To-Do

1) Fix the bug where no branches are created for a node (probably a parsing problem)

2) Speed up the algo + improve rate limit problems

4) Create Prompts for Object-Based Profiles

5) Create functions to have these conversations in mass (where I let it run and save a bunch of trees). Best if, for every real_profile, there is a saved context with my system and a saved context for out of the box ChatGPT

6) Create functions to measure the data I generated from the functinos in step 5


## Connecting to Drive and Loading Requirements


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
drive_dir = '/content/drive/MyDrive/Clarifying_Questions_GPT_Research'

In [3]:
!cd {drive_dir} && pip install -r requirements_2.txt

In [4]:
import openai

import pickle

import json

import threading

import numpy as np

import random

import re

from enum import Enum

from collections import Counter

import math

from typing import List

import copy

import signal

import time

import sys

import os

from tqdm import tqdm

import pprint

import urllib3

In [5]:
with open(drive_dir+"/openai_key.key", "r") as f:
    openai.api_key = f.read()

## Basic Functions and Decorators

In [6]:
# given a string that has a list, creates a python list with the input's contents
def str2lst(l):

  # if it is a python list as a str
  if '[' in l and ']' in l:
    # removing double spaces and newlines
    l = re.sub("\s+", " ", l)
    # removing anything that isn't the list
    m = re.search('\[(.|\s)+\]', l)
    l = l[m.start():m.end()]
    # converting string to list
    l = l.strip('][')
    l = re.split("""['"], """, l)

    # spaghetti code to dealing with this weird bug I saw before
    if len(l) == 1:
      l = l[0]
      l = re.sub("-", "", l)
      l = l.split('\n')

  # else if it is bullet points tha are...
  #numbered
  elif re.search('\d\.', l):
    # converting string to list
    l = re.findall('\d\.\s(.+)', l)

  #not numbered
  else:
    # converting string to list
    l = re.split('-', l)[1:]

  # removing extra quotation marks
  for i in range(len(l)): l[i] = re.sub("""['"]""", "", l[i])
  # removing trailing and starting spaces
  for i in range(len(l)): l[i] = l[i].strip(' ')

  return l

# function to turn a python list into a numbered list as a string
def list2numbered(lst):
  num_lst = []
  for idx in range(len(lst)): num_lst.append(f'{idx+1}. {lst[idx]}\n')
  return ''.join(num_lst)

# decorator that saves the output of func into a specific index of a lst (for multi threading)
def saveInLst(lst, idx):

  def inner(func):

    def wrap(*args, **kwargs):
      lst[idx] = func(*args, **kwargs)
      return

    return wrap

  return inner

# given a function, a list of input lists, creates a thread for each input list, running the function with different inouts in parallel. Returns a list of outputs
def multiThread(func, input_lst):
  thread_n = len(input_lst)
  threads = []

  results = [None for i in range(thread_n)]

  n = 80
  idxs = [i for i in range(thread_n )]
  batch_idxs = [idxs[i * n:(i + 1) * n] for i in range((len(idxs) + n - 1) // n )]
  # print(batch_idxs)
  for b in batch_idxs:
    print(b)
    for i in b:
      time.sleep(2) # my attempt to stop hitting the rate limit
      tr = threading.Thread(target=saveInLst(results, i)(func), args=(*input_lst[i],))
      # tr = threading.Thread(target=func, args=(*input_lst[i],))
      tr.start()
      threads.append(tr)

    for tr in threads:
      tr.join()

  return results

## Functions for Text Generation

In [7]:
# generates text using gpt-3.5 given a conversation context, returns None if its not able to generate text after a certain number of tries
def generateText(context, temp = 0.7, tries = 3, max_time = 120):
  assert isinstance(context, list), f'input is not a context list, got {type(context)}'

  resp = None

  while tries != 0:
    try:
      resp = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=context,
        temperature=temp)['choices'][0]['message']['content']

      tries = 0
    except:
      tries -= 1

  return resp


# def generateMText(context, thread_n, temp = 0.7, tries = 3):
#   threads = []

#   results = [None for i in range(thread_n)]

#   seed = 'Generate a profile of a user who is looking to travel. The profile should be comprised of a few bullet points that describe them'

#   for i in range(thread_n):
#     tr = threading.Thread(target=saveInLst(results, i)(generateText), args=(context, temp, tries,))
#     tr.start()
#     threads.append(tr)

#   for tr in threads:
#     tr.join()

#   return results

# appends a new entry to a conversation context
def appendContext(text, context, role = 'user'):
  assert role == 'user' or role == 'assistant', f'unexpected role, got: {role}'
  assert isinstance([], list), 'context has to be a list'

  if not context:
    context.append({'role': role, 'content': text})
    return

  if role == 'user':
    assert context[-1]['role'] == 'assistant', f'incompatible adjecent role, got user'
  else:
    assert context[-1]['role'] == 'user', f'incompatible adjecent role, got assistant'

  context.append({'role': role, 'content': text})

  return

# Prompts

In [8]:
def stichPrompt(prompt, var_lst):
  var_places = len(re.findall('{\S*}', prompt))
  assert var_places == len(var_lst), f'prompt has to have the same number of places for variables as variables in var_lst. Prompt has {var_places} var places and len(var_lst) = {len(var_lst)}. Here is the prompt:\n{prompt}'
  for var_idx in range(var_places):
    m = re.search('{\S*}', prompt)
    start = prompt[:m.start()]
    end = prompt[m.end():]
    prompt = f'{start}{var_lst[var_idx]}{end}'

  return prompt

# CREATE CODE THAT SERVES AS DICT OF ALL PROPMTS (WHERE THE VALUES AZRE FUNCTIONS)
stichPrompt("Here is an example prompt, with a var here: {var} and here: {}", [1,2])

'Here is an example prompt, with a var here: 1 and here: 2'

In [9]:
prompts = {'startConv': ["""You are taking an English test.\nHere is a sentence: "{i_want_sen}, but I do not know [insert word]. Help me by asking me a question at a time."\nReturn the sentence but with the correct word filled in the empty slot:"""],

           'genRealProfile': ["Generate a profile of a user. The profile should be a list of keywords about their preferences regarding {domain}."],

           'continueConvBool': ["Is the following text a question or a suggestion?/n/n{text}/n/nAnswer [Question, Suggestion]:"],

           'genUProfiles': ['Generate {number} different profiles. Each profile should be one sentence containing a sequence of keywords about their preferences regarding {domain}.'],

           'extractLastPreference': ['Here is a conversation.\nASSISTANT:\n{question}\nUSER:\n{answer}\nTASK: Summarize what we know about the user in a single sentence.\nSUMMARY:'],

           'genYProfiles': ['Here is a list of user preferences:{bullet_points}',
                            'Here is a sentence describing the preferences of a user:{sentence}\n',
                            'Create {number} different versions of this sentence, where you add many new extra preferences on top that the user might have. Each sentence should have the known preferences and also have different imagined preferences from each other:',
                            'Create different versions of this sentence, where you add many new extra preferences on top that the user might have. Each sentence should have the known preferences and also have different imagined preferences from each other:'],

           'answerQ': ['You are a character. Here is what you know about your character: {profile}\nYou are speaking to an assistant and you speak in brief sentences.\nAnswer the assistant in character.\nAssistant: {question}\nYou:'],

           'pickBranch': ["""You are the following character: {profile}\nYou are asked the following question: {question}\n\nFrom the options below, which is most likely your answer to the question?\n{bullet_points}\nRemember, you have to pick one from the list or answer "None of the answers in the list". Only return the most likely answer, followed by the number that is the place of the answer in the list.\nAnswer, Number:""",
                          "You are the following character: {profile}\nYou are asked the following question: {question}\nIs it likely that you would answer: {answer}?\n\n[Yes, No]:"],

          'makeBranches': ['Here is a question:\n"{question}"\nReturn the smallest numbered list of different possible answers to this question. Make sure that this list, althought small, covers how any user could answer this question.'],

           'makeNodes': ['Generate a python list of {node_n} questions you could possibly ask me at this point of the conversation.',
                         'Generate a python list of questions you could possibly ask me at this point of the conversation.',
                         ' Inlcude the last question you just asked in the python list.',
                         "Let's say I said the following: {answer}\n {prompt}"],


           }

# Functions for Profiles

In [10]:
# Create U Profiles
def genUProfiles(prof_num = 50, domain = 'movies'):
  prompt = stichPrompt(prompts['genUProfiles'][0], [prof_num, domain])
  #prompt = f'Generate {prof_num} different profiles. Each profile should be one sentence containing a sequence of keywords about their preferences regarding traveling.'
  return [Profile(txt, ProfileType.U) for txt in str2lst(generateText([{'role': 'user', 'content': prompt}], tries = 2))]

# Extract the last preference the user revealed in the chat context
def extractLastPreference(chat_hist):
  chat_hist = chat_hist[-2:]
  assert chat_hist[0]['role'] == 'assistant', 'chat history should have the assistant as the second to last message'
  assert chat_hist[1]['role'] == 'user', 'chat history should have the user as the last message'

  prompt = stichPrompt(prompts['extractLastPreference'][0], [chat_hist[0]['content'], chat_hist[1]['content']])
  # prompt = f'Here is a conversation.\nASSISTANT:\n{chat_hist[0]['content']}\nUSER:\n{chat_hist[1]['content']}\nTASK: Summarize what we know about the user in a single sentence.\nSUMMARY:'

  return generateText([{'role': 'user', 'content': prompt}], tries = 2)

# Create Y Profiles
def genYProfiles(preference_lst = [], prof_num = None):
  p_n = len(preference_lst)
  assert p_n != 0, 'need at least one preference in preference_lst'

  prompt = preference_lst[0]

  # if preference_lst has more than oone preferece, create a single sentence representing all of them
  if p_n > 1:
    prompt = stichPrompt(prompts['genYProfiles'][0], [list2numbered(preference_lst)])
    # prompt = f'Here is a list of user preferences:{list2numbered(preference_lst)}'
    prompt = generateText([{'role': 'user', 'content': prompt+'Create a single sentence of key words that describes all of these preferences:'}], tries = 2)

  prompt = stichPrompt(prompts['genYProfiles'][1], [prompt])
  # prompt = f'Here is a sentence describing the preferences of a user:{prompt}\n'

  # if profile number is specified, generate this specific number, otherwise let LLM decide
  if prof_num:
    prompt = stichPrompt(prompts['genYProfiles'][2], [prof_num])
    # prompt += f'Create {prof_num} different versions of this sentence, where you add many new extra preferences on top that the user might have. Each sentence should have the known preferences and also have different imagined preferences from each other:'
  else:
    prompt += prompts['genYProfiles'][3]
    # prompt += 'Create different versions of this sentence, where you add many new extra preferences on top that the user might have. Each sentence should have the known preferences and also have different imagined preferences from each other:'

  return [Profile(txt, ProfileType.Y) for txt in str2lst(generateText([{'role': 'user', 'content': prompt}], tries = 2))]

# Create a real profile
def genRealProfile(domain = 'movies'):
  prompt = stichPrompt(prompts['genRealProfile'][0], [domain])
  #prompt = f'Generate {prof_num} different profiles. Each profile should be one sentence containing a sequence of keywords about their preferences regarding traveling.'
  return Profile(generateText([{'role': 'user', 'content': prompt}], tries = 2), ProfileType.R)

# Tree Structure

In [103]:
# Enum of Profile types
class ProfileType(Enum):
  U = 0
  Y = 1
  R = 2

# Class representing a user profile
class Profile:
  def __init__(self, text: str, p_type: ProfileType):
    self.text = text
    self.p_type = p_type
    return

  # # makes this Proifle into a UProfile
  # def makeUProfile(self):
  #   self.p_type = ProfileType.U
  #   return

  # # makes this Proifle into a YProfile
  # def makeYProfile(self, context):
  #   self.p_type = ProfileType.Y
  #   return

  # answers question q as the user the profile describes, returns string
  @staticmethod
  def answerQ(self, q):
    prompt = stichPrompt(prompts['answerQ'][0], [self.text, q])
    # prompt = f'You are a character. Here is what you know about your character: {self.text}\nYou are speaking to an assistant and you speak in brief sentences.\nAnswer the assistant in character.\nAssistant: {q}\nYou:'
    return generateText([{'role': 'user', 'content': prompt}])

  # picks branch that best fits profile, returns int that is index of picked branch in the branches list or None
  @staticmethod
  def pickBranch(self, node):
    prompt = stichPrompt(prompts['pickBranch'][0], [self.text, node.question, list2numbered(node.branches)])
    # prompt = f"""You are the following character: {self.text}\nYou are asked the following question: {node.question}\n\nFrom the options below, which is most likely your answer to the question?\n{list2numbered(node.branches)}\nRemember, you have to pick one from the list or answer "None of the answers in the list". Only return the most likely answer, followed by the number that is the place of the answer in the list.\nAnswer, Number:"""
    txt = generateText([{'role': 'user', 'content': prompt}], temp=1.0)

    # trying to find a number in the answer generated
    n_m = re.search('\d', txt)

    if n_m:
      # if the number chosen is not an index to the branches list, return None
      branch_idx = int(txt[n_m.start():n_m.end()]) - 1
      if branch_idx >= len(node.branches):
        return

      # if given an appropriate answer, make sure the answer shouldn't be neither
      prompt = stichPrompt(prompts['pickBranch'][1], [self.text, node.question, node.branches[branch_idx]])
      # prompt = f"You are the following character: {self.text}\nYou are asked the following question: {node.question}\nIs it likely that you would answer: {txt[t_m.start():t_m.end()]}?\n\n[Yes, No]:"

      if 'yes' in generateText([{'role': 'user', 'content': prompt}]).lower():
        return branch_idx

    # if no number was given, or if the number was not an appropriate answer, return None
    return

# Class representing a node in a layer of the "decision tree"
class Node:
  def __init__(self, q : str):
    # question that represents this node
    self.question = q
    # possible branches/answers to this node/question
    self.branches = None
    self.tst_b = None
    # map of profile index (from given profile_lst input in splitProfiles func) to branch index
    self.profile_branch_map = None
    # entropy and info gain for this Node
    self.entropy = None
    self.info_gains = None

    return

  # generates branches for this node
  @staticmethod
  def makeBranches(self):
    assert self.question != None, "self.question cannot be None"

    prompt = stichPrompt(prompts['makeBranches'][0], [self.question])
    # prompt = f'Here is a question:\n"{self.question}"\nReturn a concise list of different possible answers to the question.'
    self.tst_b = generateText([{'role': 'user', 'content': prompt}], tries = 2, temp = 1.0)
    self.branches = str2lst(self.tst_b)
    return

  # splits profiles into the different branches, then creates a mapping from profile index to branch index
  @staticmethod
  def splitProfiles(self, profile_lst : List[Profile]):
    assert self.branches != None, "self.branches cannot be None"
    assert len(profile_lst) != 0, "self.profile_lst cannot be empty"

    # each profile picks branch that is best fit, returns list of int
    # saving what profile picked what branch in self.profile_branch_map
    self.profile_branch_map = multiThread(Profile.pickBranch, [[p, self] for p in profile_lst])

    # editing self.branches and self.profile_branch_map if a profile picked no Branches
    has_none = False
    for idx in range(len(self.profile_branch_map)):
      if self.profile_branch_map[idx] == None:
        self.profile_branch_map[idx] = len(self.branches)
        has_none = True

    if has_none:
      self.branches.append("NONE OF THE ABOVE")

    return

  # calculating info gain
  def clalcInfoGain(self, profile_lst : List[Profile]):
    # number of total profiles
    total_profiles_n = len(profile_lst)
    # creating a set of classes
    classes = set()
    for p in profile_lst:
      classes.add(p.p_type)
    # map from branch index to count of profiles that picked this branch
    branch_count = Counter(self.profile_branch_map)
    # map from branch index to possible node entropy
    branch_entropy = []

    # if there is only one class, treat every profile as its own class
    if len(classes) == 1:
      # calculating this node's entropy
      self.entropy = -1*math.log2(1/total_profiles_n)

      # calculating the entropy of each branch
      for b_idx in range(len(self.branches)):
        if branch_count[b_idx] != 0: branch_entropy.append(-1*math.log2(1/branch_count[b_idx]))
        else: branch_entropy.append(0)

    # otherwise
    else:
      # calculating this node's entropy
      self.entropy = 0
      for c in Counter([p.p_type.value for p in profile_lst]).values(): self.entropy -= (c/total_profiles_n)*math.log2(c/total_profiles_n)

      # calculating the entropy of each branch
      assert 1==0

    # calculating information gain
    branch_entropy_weighted_sum = 0
    for b_idx in range(len(self.branches)):
      branch_entropy_weighted_sum += (branch_count[b_idx]/total_profiles_n) * branch_entropy[b_idx]

    self.info_gains = self.entropy - branch_entropy_weighted_sum
    return


# Class representing a layer of the "decision tree"
class Layer:
  def __init__(self, context):
    # current context
    self.context = context
    # list of nodes, each representing a possible question for this layer of the conversation
    self.nodes = None

    # list of Profile
    self.profiles = []
    # list of the preferences from the user we are aware of
    self.known_preferences = []

    # index of the node with the highest information gain
    self.best_node_idx = None

    # stores next layer
    self.next = None
    # stores previous layer
    self.prev = None

    return

  # Extract the last preference the user revealed in the chat context
  def extractLastPreference(self):
    assert len(self.context) >= 3, 'context needs to have at least three entries (the seed prompt + one for the user and one for the assistant)'

    chat_hist = self.context[-2:]
    assert chat_hist[0]['role'] == 'assistant', 'chat history should have the assistant as the second to last message'
    assert chat_hist[1]['role'] == 'user', 'chat history should have the user as the last message'

    prompt = stichPrompt(prompts['extractLastPreference'][0], [chat_hist[0]['content'], chat_hist[1]['content']])
    # prompt = f'Here is a conversation.\nASSISTANT:\n{chat_hist[0]['content']}\nUSER:\n{chat_hist[1]['content']}\nTASK: Summarize what we know about the user in a single sentence.\nSUMMARY:'

    self.known_preferences.append(generateText([{'role': 'user', 'content': prompt}], tries = 2))
    return

  # creating U Profiles
  def makeUProfiles(self, prof_num = None, domain = 'movies'):
    # IF THERE ARE NO KNOWN PREFERENCES
    if not self.known_preferences:
      if prof_num: prompt = stichPrompt(prompts['makeUProfiles'][0], [prof_num, domain])
      else: prompt = stichPrompt(prompts['makeUProfiles'][1], [domain])
      #prompt = f'Generate {prof_num} different profiles. Each profile should be one sentence containing a sequence of keywords about their preferences regarding traveling.'
      self.profiles = [Profile(txt, ProfileType.U) for txt in str2lst(generateText([{'role': 'user', 'content': prompt}], tries = 2))]
      return

    # IF THERE ARE KNOWN PREFERENCES, incoroporate them into the profiles
    p_n = len(self.known_preferences)
    prompt = self.known_preferences[0]

    # if preference_lst has more than one preferece, create a single sentence representing all of them
    if p_n > 1:
      prompt = stichPrompt(prompts['makeUProfiles'][2], [list2numbered(self.known_preferences)])
      # prompt = f'Here is a list of user preferences:{list2numbered(preference_lst)}'
      prompt = generateText([{'role': 'user', 'content': prompt+'Create a single sentence of key words that describes all of these preferences:'}], tries = 2)

    prompt = stichPrompt(prompts['makeUProfiles'][3], [prompt])
    # prompt = f'Here is a sentence describing the preferences of a user:{prompt}\n'

    # if profile number is specified, generate this specific number, otherwise let LLM decide
    if prof_num:
      prompt += stichPrompt(prompts['makeUProfiles'][4], [prof_num, domain])
      # prompt += f'Create {prof_num} different versions of this sentence, where you add many new extra preferences on top that the user might have. Each sentence should have the known preferences and also have different imagined preferences from each other:'
    else:
      prompt += stichPrompt(prompts['makeUProfiles'][5], [domain])
      # prompt += 'Create different versions of this sentence, where you add many new extra preferences on top that the user might have. Each sentence should have the known preferences and also have different imagined preferences from each other:'

    pprint.pprint(prompt)
    self.profiles = [Profile(txt, ProfileType.U) for txt in str2lst(generateText([{'role': 'user', 'content': prompt}], tries = 2))]
    return

  # creating nodes, hence possible questions for this layer
  def makeNodes(self, node_n = None):
    # making a deep copy of the context so it is unchanged
    context = copy.deepcopy(self.context)

    # making the base prompt
    if node_n:
      prompt = stichPrompt(prompts['makeNodes'][0], [node_n])
      # prompt = f'Generate a python list of {node_n} questions you could possibly ask me at this point of the conversation.'
    else:
      prompt = prompts['makeNodes'][1]
      # prompt = 'Generate a python list of questions you could possibly ask me at this point of the conversation.'

    if context[-1]['role'] == 'assistant':
      prompt += prompts['makeNodes'][2]
      # prompt += ' Inlcude the last question you just asked in the python list.'
    else:
      prompt = stichPrompt(prompts['makeNodes'][3], [context.pop()['content'], prompt])
      # prompt = f"Let's say I said the following: {context.pop()['content']}\n {prompt}"


    # appending the prompt to context
    appendContext(prompt, context, 'user')

    # generating the questions and storing them as a python list, then initializing nodes
    self.nodes = [Node(q) for q in str2lst(generateText(context, tries = 2))]
    return

  # creates branches for each Node in self.nodes
  def makeBranches(self):
    assert self.nodes != None, "self.nodes cannot be None"
    multiThread(Node.makeBranches, [[n] for n in self.nodes])
    return

  # create a mapping from profiles to branches for each node
  def splitProfiles(self):
    assert self.nodes != None, "self.nodes cannot be None"
    assert len(self.profiles) != 0, "self.profiles cannot be empty"
    # multiThread(Node.splitProfiles, [[n, self.profiles] for n in self.nodes])
    # return
    for n in self.nodes:
      n.splitProfiles(n, self.profiles)
    return

  # calculates info gain for every node and saves index of best node
  def calcInfoGain(self):
    for n in self.nodes: n.clalcInfoGain(self.profiles)
    ig = [n.info_gains for n in self.nodes]
    self.best_node_idx = ig.index(max(ig))

    return

  # ## NEED TO IMPLEMENT THIS FOR NEW CONCENPTION OF NODE (AS WELL AS ALL THE OTHER FUNCTIONS)
  # # creates branches using profiles by calling the ProfileGroup pickbranch(self, node) method for the ProfileGroup in index profile_group_idx in self.profile_groups
  # def makeBranchesP(self, profile_group_idx = None):
  #   if profile_group_idx and len(self.profiles) <= profile_group_idx:
  #     print(f'self.profiles does not have index {profile_group_idx}')
  #     return

  #   if profile_group_idx and len(self.profiles[profile_group_idx]) == 0:
  #     print(f'no profiles exists in index {profile_group_idx }')
  #     return

  #   if profile_group_idx:
  #     b_maker_ps = self.profiles[profile_group_idx]
  #   else:
  #     b_maker_ps = [p for p_group in self.profiles for p in p_group]

  #   self.b = multiThread(Profile.answerQ, [[p, self.q] for p in b_maker_ps])
  #   return

  # # aggregate branches that are the same, sets self.b to a list of strings GIVEN self.b is not none
  # def aggregateBranches(self):
  #   if not self.b:
  #     print('this node has no branches')
  #     return

  #   self.b = list(dict.fromkeys(self.b))
  #   prompt = f'You are taking an English test.\nHere is a question: "{self.q}"\nHere is a python list of possible answers to the question: {self.b}\nThe current python list may contain answers that are worded differently, but have the same meaning. Remove all the duplicate answers and return a python list of unique answers:'
  #   self.b = str2lst(generateText([{'role': 'user', 'content': prompt}]))
  #   return

# Conversartion Functions

In [12]:
def startConv(i_want_sen = 'watch a movie'):
  prompt = stichPrompt(prompts['startConv'][0], [i_want_sen])
  # prompt = f"""You are taking an English test.\nHere is a sentence: "{i_want_sen}, but I do not know [insert word]. Help me by asking me a question at a time."\nReturn the sentence but with the correct word filled in the empty slot:"""
  prompt = generateText([{'role': 'user', 'content': prompt}]).strip(""" '" """)

  context = [{'role': 'user', 'content': prompt}]
  # appendContext(generateText([{'role': 'user', 'content': prompt}]), context, 'assistant')

  return context

def continueConvBool(chat_hist, i_want_sen = 'watch a movie'):
  assert chat_hist[-1]['role'] == 'assistant', 'chat history should have the assistant as the last message'
  prompt = stichPrompt(prompts['continueConvBool'][0], [i_want_sen, chat_hist[-1]['content']])

  return 'ask' in generateText([{'role': 'user', 'content': prompt}]).lower()

In [13]:
def convStep(context):
  return

#Main

In [130]:
prompts = {'startConv': ["""You are taking an English test.\nHere is a sentence: "I want to {i_want_sen}, but I do not know [insert word]. Help me by asking me a question at a time."\nReturn the sentence but with the correct word filled in the empty slot:"""],

           'genRealProfile': ["Generate an example profile of a possible user. The profile should be a short sentence of keywords about their personal preferences regarding {domain}. Make the profile very specific:"],

           'continueConvBool': ["An assistant is trying to help a user make a decision. The user wants to {i_want_sen}. Is the following text the assistant asking about the user's preferences or is it a suggestion?/n/n{text}/n/nAnswer [Asking, Suggestion]:"],

           'makeUProfiles': ['Generate {number} different profiles. Each profile should be one sentence containing a sequence of keywords about their preferences regarding {domain}.',
                             'Generate different profiles. Each profile should be one sentence containing a sequence of keywords about their preferences regarding {domain}.',
                             'Here is a list of known user preferences: {bullet_points}',
                             'Here is a sentence describing the known preferences of a user: {sentence}\n',
                             'Create {number} different versions of this sentence, where you add many new extra preferences regarding {domain} that the user might have. Each sentence should have the following structure,"known preferences + their own imagined preferences". Make sure the extra imagined preferences you add to each version are (1) compatible with the known preferences of the user and (2) are different from the imagined preferences from the other versions:',
                             'Create different versions of this sentence, where you add many new extra preferences regarding {domain} that the user might have. Each sentence should have the following structure,"known preferences + their own imagined preferences". Make sure the extra imagined preferences you add to each version are (1) compatible with the known preferences of the user and (2) are different from the imagined preferences from the other versions:'],

           'genUProfiles': ['Generate {number} different profiles. Each profile should be one sentence containing a sequence of keywords about their preferences regarding {domain}.'],

           'extractLastPreference': ['Here is a conversation.\nASSISTANT:\n{question}\nUSER:\n{answer}\nTASK: Summarize what we know about the user in a single sentence.\nSUMMARY:'],

           'genYProfiles': ['Here is a list of user preferences:{bullet_points}',
                            'Here is a sentence describing the preferences of a user:{sentence}\n',
                            'Create {number} different versions of this sentence, where you add many new extra preferences on top that the user might have. Each sentence should have the known preferences and also have different imagined preferences from each other:',
                            'Create different versions of this sentence, where you add many new extra preferences on top that the user might have. Each sentence should have the known preferences and also have different imagined preferences from each other:'],

           'answerQ': ['You are a character. Here is what you know about your character: {profile}\nYou are speaking to an assistant and you speak in brief sentences.\nAnswer the assistant in character.\nAssistant: {question}\nYou:'],

           'pickBranch': ["""You are the following character: {profile}\nYou are asked the following question: {question}\n\nFrom the options below, which is most likely your answer to the question?\n{bullet_points}\nRemember, you have to pick one from the list or answer "None of the answers in the list". Only return the most likely answer, followed by the number that is the place of the answer in the list.\nAnswer, Number:""",
                          "You are the following character: {profile}\nYou are asked the following question: {question}\nIs it likely that you would answer: {answer}?\n\n[Yes, No]:"],

          'makeBranches': ['Here is a question:\n"{question}"\nReturn the smallest numbered list of different possible answers to this question. Make sure that this list, althought small, covers how any user could answer this question.\nList of answers:'],

           'makeNodes': ['Generate a python list of {node_n} questions you could possibly ask me at this point of the conversation.',
                         'Generate a python list of questions you could possibly ask me at this point of the conversation.',
                         ' Inlcude the last question you just asked in the python list.',
                         "Let's say I said the following: {answer}\n {prompt}"],


           }

In [104]:
# initialize layer
layer = Layer(startConv())

In [105]:
# generate u profiles
layer.makeUProfiles(10)

In [109]:
# generate the real profile
real_profile = genRealProfile()

In [110]:
# printing real profile
pprint.pprint(real_profile.text)
# printing their response to the following question as a quick little test
pprint.pprint(real_profile.answerQ(real_profile, 'Are you interested in watching a movie like "Jumanji"?'))

('Action movie enthusiast, prefers high-octane stunts, intense fight scenes, '
 'and explosive special effects; enjoys movies with fast-paced plots and '
 'badass protagonists.')
'Nah, not really my style.'


In [111]:
# creating nodes in layer, hence possible questions
layer.makeNodes(3)

In [112]:
[n.question for n in layer.nodes]

['What genre of movies do you usually enjoy?',
 'Do you prefer watching recent releases or classic films?',
 'Are you in the mood for a comedy, drama, or action movie?']

In [113]:
# creating the branches for each of the nodes (without using profiles)
layer.makeBranches()

[0, 1, 2]


In [114]:
pprint.pprint([n.branches for n in layer.nodes])

[['Action',
  'Comedy',
  'Drama',
  'Romance',
  'Horror',
  'Science fiction',
  'Thriller',
  'Adventure',
  'Animation',
  'Fantasy'],
 ['Recent releases', 'Classic films'],
 ['Yes (indicating that the user is open to any genre)',
  'No (indicating that the user is not in the mood for any genre)',
  'Comedy (indicating that the user specifically wants to watch a comedy)',
  'Drama (indicating that the user specifically wants to watch a drama)',
  'Action (indicating that the user specifically wants to watch an action '
  'movie)']]


In [115]:
# layer.nodes[0].tst_b

In [116]:
# split the profiles
layer.splitProfiles()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [117]:
for n in layer.nodes: print(n.profile_branch_map)

[5, 2, 8, 4, 2, 0, 2, 6, 3, 5]
[0, 1, 1, 1, 2, 2, 1, 1, 1, 2]
[4, 3, 2, 5, 3, 4, 3, 3, 2, 2]


In [118]:
for n_idx in range(len(layer.nodes)):
  print(f'////////// {layer.nodes[n_idx].question} //////////\n')
  for p_idx in range(len(layer.profiles)):
    print(f'{layer.profiles[p_idx].text}')
    choice = layer.nodes[n_idx].profile_branch_map[p_idx]
    print(f'{layer.nodes[n_idx].branches[choice]}\n')
    # if choice is not None: print(f'{layer.nodes[n_idx].branches[choice]}\n')
    # else: print('None\n')

////////// What genre of movies do you usually enjoy? //////////

Sarah: Avid moviegoer who enjoys thrilling action-packed blockbusters, mind-bending sci-fi flicks, and heartwarming romantic comedies.
Science fiction

Tom: A cinephile with a penchant for classic black and white films, thought-provoking independent dramas, and critically acclaimed foreign movies.
Drama

Emily: An animation enthusiast who adores enchanting animated films, whimsical fantasy adventures, and heartwarming tales of friendship and family.
Animation

Alex: A horror aficionado who revels in spine-chilling supernatural thrillers, terrifying psychological horrors, and gory slasher movies that keep them on the edge of their seat.
Horror

Mia: A fan of heartwrenching tearjerkers, emotionally charged dramas, and thought-provoking films that explore complex human relationships and societal issues.
Drama

Jake: A lover of high-octane sports films, adrenaline-pumping racing movies, and inspiring underdog stories that sh

In [119]:
# Calculate InfoGain for each node
layer.calcInfoGain()

In [120]:
for n in layer.nodes:
  print(n.info_gains)

print(layer.nodes[layer.best_node_idx].question)

2.6464393446710153
1.295461844238322
1.8464393446710152
What genre of movies do you usually enjoy?


Repeat for a round here -------------------------------------------------

In [171]:
# Real Profile answers question
ans = real_profile.answerQ(real_profile, layer.nodes[layer.best_node_idx].question)
ans

"As long as it has high-octane action, I'm open to watching a foreign superhero film."

In [172]:
# creating the context of the next layer
next_context = copy.deepcopy(layer.context)
appendContext(layer.nodes[layer.best_node_idx].question, next_context, 'assistant')
appendContext(ans, next_context, 'user')
pprint.pprint(next_context)

# creating the next layer
layer.next = Layer(next_context)
layer.next.prev = layer

# setting layer to be layer.next
layer = layer.next

[{'content': 'I want to watch a movie, but I do not know what. Help me by '
             'asking me a question at a time.',
  'role': 'user'},
 {'content': 'What genre of movies do you usually enjoy?', 'role': 'assistant'},
 {'content': 'I love action movies.', 'role': 'user'},
 {'content': 'Are you in the mood for a fast-paced action film or something '
             'more suspenseful?',
  'role': 'assistant'},
 {'content': 'Definitely in the mood for a fast-paced action film.',
  'role': 'user'},
 {'content': 'Would you like the action movie to have a comedic element or be '
             'more serious in tone?',
  'role': 'assistant'},
 {'content': 'Definitely more serious in tone.', 'role': 'user'},
 {'content': 'Are you interested in watching a superhero action movie or '
             'something more grounded in reality?',
  'role': 'assistant'},
 {'content': 'Definitely a superhero action movie.', 'role': 'user'},
 {'content': 'Are you open to watching a foreign superhero film or d

In [173]:
# see if it gives a suggestion
pprint.pprint(layer.context[:])
text = generateText(layer.context[:])
print(text + '\n')

continue_bool_context = copy.deepcopy(layer.context[:]) + [{'content': text, 'role': 'assistant'}]
continue_bool = continueConvBool(continue_bool_context)
print(str(continue_bool) + '\n')

 # if it gives a suggestion, see if the real profile likes it
if not continue_bool: print(real_profile.answerQ(real_profile, text))

[{'content': 'I want to watch a movie, but I do not know what. Help me by '
             'asking me a question at a time.',
  'role': 'user'},
 {'content': 'What genre of movies do you usually enjoy?', 'role': 'assistant'},
 {'content': 'I love action movies.', 'role': 'user'},
 {'content': 'Are you in the mood for a fast-paced action film or something '
             'more suspenseful?',
  'role': 'assistant'},
 {'content': 'Definitely in the mood for a fast-paced action film.',
  'role': 'user'},
 {'content': 'Would you like the action movie to have a comedic element or be '
             'more serious in tone?',
  'role': 'assistant'},
 {'content': 'Definitely more serious in tone.', 'role': 'user'},
 {'content': 'Are you interested in watching a superhero action movie or '
             'something more grounded in reality?',
  'role': 'assistant'},
 {'content': 'Definitely a superhero action movie.', 'role': 'user'},
 {'content': 'Are you open to watching a foreign superhero film or d

In [163]:
# creating nodes in new layer, hence possible questions
layer.makeNodes()
[n.question for n in layer.nodes]

['Do you have a specific superhero in mind or are you open to any?',
 'Would you prefer a standalone superhero film or one that is part of a larger cinematic universe?',
 'Are you looking for a recent release or a classic superhero movie?',
 'Are there any particular actors or actresses you enjoy watching in action films?',
 'How important is the storyline and character development compared to the action sequences for you?',
 'Would you like the movie to have a darker and grittier tone or a more light-hearted and optimistic tone?',
 'Are you interested in a superhero origin story or a film that focuses on a heros battle against a specific villain?',
 'How long of a runtime are you comfortable with for the movie?',
 'Are you open to watching a foreign superhero film or do you prefer English-language movies?',
 'Is there a specific director or film franchise that you are a fan of within the superhero genre?']

In [164]:
# creating the branches for each of the nodes (without using profiles)
layer.makeBranches()
pprint.pprint([n.branches for n in layer.nodes])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[['Yes, I have a specific superhero in mind.',
  'No, I am open to any superhero.'],
 ['Standalone superhero film.',
  'Cinematic universe.',
  'It depends on the specific superhero and storyline.'],
 ['Recent release', 'Classic superhero movie'],
 ['Yes',
  'No',
  'I do not watch action films',
  'I enjoy watching various actors and actresses in action films'],
 ['The storyline and character development are highly important, and I value '
  'them more than the action sequences.',
  'Both the storyline/character development and action sequences are equally '
  'important to me.',
  'I prioritize the action sequences over the storyline and character '
  'development.',
  'I dont have a preference or opinion on this matter.'],
 ['Darker and grittier tone', 'Light-hearted and optimistic tone'],
 ['Origin story', 'Battle against a specific villain'],
 ['90 minutes', '2 hours', '3 hours'],
 ['Yes, I am open to watching a foreign superhero film.',
  'No, I pre

In [165]:
# extracting the most recent preference discovered
if layer.prev: layer.known_preferences = layer.prev.known_preferences

layer.extractLastPreference()
layer.known_preferences

['The user enjoys action movies.',
 'The user prefers fast-paced action films over suspenseful ones.',
 'The user prefers action movies that have a serious tone rather than a comedic element.',
 'The user prefers watching superhero action movies over movies grounded in reality.']

In [166]:
# generate u profiles
layer.makeUProfiles(10)
for p_idx in range(len(layer.profiles)): print(f'{p_idx} : {layer.profiles[p_idx].text}\n')


('Here is a sentence describing the known preferences of a user: User prefers '
 'fast-paced, serious, superhero action movies.\n'
 'Create 10 different versions of this sentence, where you add many new extra '
 'preferences regarding movies that the user might have. Each sentence should '
 'have the following structure,"known preferences + their own imagined '
 'preferences". Make sure the extra imagined preferences you add to each '
 'version are (1) compatible with the known preferences of the user and (2) '
 'are different from the imagined preferences from the other versions:')
0 : The user prefers fast-paced, serious, superhero action movies and also enjoys intense chase scenes and thrilling plot twists.

1 : Users known preferences include fast-paced, serious, superhero action movies, but they also have a soft spot for visually stunning cinematography and complex character development.

2 : Apart from their love for fast-paced, serious, superhero action movies, the user also app

In [168]:
# split the profiles
layer.splitProfiles()
print()
for n in layer.nodes: print(n.profile_branch_map)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

[0, 2, 1, 1, 1, 1, 1, 2, 1, 1]
[1, 2, 2, 2, 1, 1, 1, 1, 2, 1]
[0, 0, 2, 0, 0, 0, 0, 0, 0, 0]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[4, 0, 0, 0, 0, 0, 1, 0, 0, 1]
[0, 0, 2, 0, 0, 1, 1, 0, 0, 1]
[1, 1, 2, 1, 1, 1, 1, 1, -1, 1]
[1, 1, 3, 1, 1, 3, 1, 1, 1, 1]
[3, 3, 2, 2, 2, 0, 0, 0, 3, 3]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [169]:
for n_idx in range(len(layer.nodes)):
  print(f'////////// {layer.nodes[n_idx].question} //////////\n')
  for p_idx in range(len(layer.profiles)):
    print(f'{layer.profiles[p_idx].text}')
    choice = layer.nodes[n_idx].profile_branch_map[p_idx]
    print(f'{layer.nodes[n_idx].branches[choice]}\n')
    # if choice is not None: print(f'{layer.nodes[n_idx].branches[choice]}\n')
    # else: print('None\n')

////////// Do you have a specific superhero in mind or are you open to any? //////////

The user prefers fast-paced, serious, superhero action movies and also enjoys intense chase scenes and thrilling plot twists.
Yes, I have a specific superhero in mind.

Users known preferences include fast-paced, serious, superhero action movies, but they also have a soft spot for visually stunning cinematography and complex character development.
NONE OF THE ABOVE

Apart from their love for fast-paced, serious, superhero action movies, the user also appreciates movies with thought-provoking dialogues and deep philosophical themes.
No, I am open to any superhero.

In addition to their preference for fast-paced, serious, superhero action movies, the user has a fondness for movies that explore the concept of time travel and alternate realities.
No, I am open to any superhero.

Along with their passion for fast-paced, serious, superhero action movies, the user thoroughly enjoys movies with mind-bending

In [170]:
layer.calcInfoGain()

for n in layer.nodes:
  print(n.info_gains)

print(layer.nodes[layer.best_node_idx].question)

1.1567796494470395
0.9709505944546684
0.4689955935892809
0.0
1.1567796494470395
1.295461844238322
0.9219280948873618
0.7219280948873616
1.5709505944546684
0.0
Are you open to watching a foreign superhero film or do you prefer English-language movies?


# OLD CODE

In [ ]:
def choose_with_idk(options, p_idk = 0.2):
  return random.choices([random.choice(options), "I don't know"], weights = [1-p_idk, p_idk], k=1)[0]

In [ ]:
def choose_branch(q, branches, profiles):
  ans = []

  for p in profiles:
    prompt = f"""You are the following character: {p}\nYou are asked the following question: {q}\n\nFrom the options below, which is most likely your answer to the question?\n{list2numbered(branches)}\nRemember, you have to pick one from the list. Only return the most likely answer, followed by the number that is the place of the answer in the list.\nAnswer, Number:"""
    txt = generateText([{'role': 'user', 'content': prompt}])
    m = re.search('\d', txt)
    if m:
      s = m.start()
      e = m.end()
      ans.append(int(txt[s:e]))
    else: ans.append(0)


  return ans

In [ ]:
def genAnswP(profile, q):
  prompt = f'You are a character. Here is what you know about your character: {profile}\nYou are speaking to an assistant and you speak in brief sentences.\nAnswer the assistant in character.\nAssistant: {q}\nYou:'
  return generateText([{'role': 'user', 'content': prompt}])

def genAns(q):
  prompt = f'Here is a question:\n"{q}"\nReturn a python list with all the possible answers to the question:'
  txt = generateText([{'role': 'user', 'content': prompt}], tries = 2)
  return str2lst(txt)

def aggregateAns(ans_lst, q = None):
  print('aggregating')
  ans_lst = list(dict.fromkeys(ans_lst))
  #prompt = f'Here is a list of preferences: {ans_lst}\nRemove all the answers that mean the same thing and return a list of unique answers:' BEST
  # prompt = f'Here is a list of preferences: {ans_lst}\nRemove all the answers that are too similar and return a small list of unique answers:'
  if q:
    #prompt = f'You will be given a python list that has possible answers to the question "{q}"\nYour task is to remove all the answers that mean the same thing and return a small list of unique answers.\nHere is the python list: {ans_lst}'
    prompt = f'You are taking an English test.\nHere is a question: "{q}"\nHere is a python list of possible answers to the question: {ans_lst}\nThe current python list contains answers that are worded differently, but contain the same meaning. Remove all the duplicate answers and return a python list of unique answers:'
  else:
    prompt = f'Here is a list of preferences: {ans_lst}\nRemove all the answers that mean the same thing and return a list of unique answers:'

  return str2lst(generateText([{'role': 'user', 'content': prompt}]))

In [ ]:
# Class representing a group of profiles
class ProfileGroup:
  def __init__(self, preference_lst = None):
    self.preference_lst = preference_lst
    self.profiles = []
    # if preference_lst:
    #   self.profiles = self.makeYProfiles()
    #   self.pType = ProfileType.Y
    # else:
    #   self.profiles = self.makeUProfiles()
    #   self.pType = ProfileType.U
    return

  # each profile answers question q, returns list of strings
  def answerQ(self, q):
    return multiThread(Profile.answerQ, [[p, self.q] for p in self.profiles])

  # each profile picks branch that is best fit, returns list of int
  def pickBranch(self, node):
    return multiThread(Profile.pickBranch, [[p, node] for p in self.profiles])

  def makeUProfiles(self, prof_num):
    prompt = f'Generate {prof_num} different profiles. Each profile should be one sentence containing a sequence of keywords about their preferences regarding traveling.'
    txt = generateText([{'role': 'user', 'content': prompt}], tries = 2)
    self.profiles.extend(str2lst(txt))
    return

  def makeYProfiles(self):
    p_n = len(self.preference_lst)
    prompt = self.preference_lst[0]

    # if preference_lst has more than one preferece, create a single sentence representing all of them
    if p_n > 1:
      prompt = f'Here is a list of user preferences:{list2numbered(self.preference_lst)}'
      prompt = generateText([{'role': 'user', 'content': prompt+'Create a single sentence of key words that describes all of these preferences:'}], tries = 2)

    prompt = f'Here is a sentence describing the preferences of a user:{prompt}\n'

    # if profile number is specified, generate this specific number, otherwise let LLM decide
    prompt += f'Create {self.prof_num} different versions of this sentence, where you add many new extra preferences on top that the user might have. Each sentence should have the known preferences and also have different imagined preferences from each other:'
    # prompt += 'Create different versions of this sentence, where you add many new extra preferences on top that the user might have. Each sentence should have the known preferences and also have different imagined preferences from each other:'

    return str2lst(generateText([{'role': 'user', 'content': prompt}], tries = 2))

In [ ]:
qs_ans = [[] for q in range(len(p_qs))]
for idx, a in enumerate(ans): qs_ans[idx%len(p_qs)].append(a)
print(len(qs_ans))
print(len(qs_ans[0]))

In [ ]:
qs_ans[0]

In [ ]:
# aggregating answers
branches = [[] for q in range(len(p_qs))]
for i, ans in enumerate(qs_ans):
  branches[i] = aggregateAns(ans)

for b in branches:
  print(len(b))
pprint.pprint(branches)

In [ ]:
branches = [[] for q in range(len(p_qs))]
for i, ans in enumerate(qs_ans):
  branches[i] = aggregateAns(ans, p_qs[i])

for b in branches:
  print(len(b))
pprint.pprint(branches)